<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/myProspects_200D_50D_SMA_Strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
#from google.colab import drive
#drive.mount('/content/drive')

#mypf = pd.read_csv('/content/drive/My Drive/data/stocks/myProspectsScrips.csv')

path_mypf = 'https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/'

mypf = pd.read_csv(path_mypf + 'myProspectsScrips.csv')

mypf['Symbol'].values

array(['3MINDIA', '5PAISA', 'AARTIIND', 'AAVAS', 'ABBOTINDIA', 'ABSLAMC',
       'ACC', 'AETHER', 'AKZOINDIA', 'ALKYLAMINE', 'AMBUJACEM',
       'ANGELONE', 'APEX', 'APLLTD', 'APOLLOHOSP', 'ARVINDFASN', 'ARE&M',
       'ASIANPAINT', 'ASIANTILES', 'ASTRAZEN', 'AUBANK', 'AUROPHARMA',
       'AVANTIFEED', 'AWL', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV',
       'BAJAJHIND', 'BAJAJHLDNG', 'BAJFINANCE', 'BALKRISIND',
       'BANDHANBNK', 'BANKBARODA', 'BANKINDIA', 'BATAINDIA', 'BAYERCROP',
       'BERGEPAINT', 'BHARTIARTL', 'BIOCON', 'BIRLAMONEY', 'BLUESTARCO',
       'BOROLTD', 'BOSCHLTD', 'BPCL', 'BRITANNIA', 'BSOFT', 'CAMPUS',
       'CAMS', 'CANBK', 'CAPLIPOINT', 'CASTROLIND', 'CDSL', 'CENTRALBK',
       'CERA', 'CHOLAFIN', 'CIPLA', 'CLEAN', 'COALINDIA', 'COFFEEDAY',
       'COLPAL', 'CROMPTON', 'CUB', 'DABUR', 'DCBBANK', 'DEEPAKNTR',
       'DEN', 'DIVISLAB', 'DIXON', 'DLF', 'DMART', 'DREAMFOLKS',
       'DRREDDY', 'EASEMYTRIP', 'EICHERMOT', 'EMAMILTD', 'EQUITASBNK',
       'ERIS', 'EXIDE

In [2]:
mypf['Category'].unique()

array(['MISC', 'FINANCE', 'CHEMICALS', 'PHARMA', 'CEMENT', 'PAINTS',
       'HEALTHCARE', 'AUTO', 'CERAMICS', 'BANKS', 'FMCG', 'SUGAR',
       'TYRES', 'FOOTWEAR', 'TELECOM', 'AC', 'GLASS', 'OIL', 'IT',
       'MINING', 'HOTELS', 'ELECTRICAL', 'ENTERTAINMENT', 'REALTY',
       'TRAVEL', 'CABLES', 'GAS', 'INSURANCE', 'TEXTILES', 'METALS',
       'REFINERIES', 'AIRLINES', 'STEEL', 'POWER', 'RESTAURANTS',
       'ENGINEERING', 'BREWERIES', 'MEDIA', 'APPARELS', 'JEWELLERY',
       'BEARINGS', 'DURABLES', 'PAPER'], dtype=object)

In [3]:
import yfinance as yf
import pandas as pd

class StockSMADeviationFilter:

    def __init__(self, stock_list, start_date='2018-01-01', end_date=None):
        self.stock_list = stock_list
        self.start_date = start_date
        self.end_date = end_date
        self.filtered_stocks = []

    def fetch_data(self, ticker):
        """Fetch stock data from Yahoo Finance."""
        try:
            data = yf.download(ticker + '.NS', start=self.start_date, end=self.end_date)
            return data
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            return None

    def calculate_smas(self, data):
        """Calculate the 50-day and 200-day SMAs."""
        data['50_SMA'] = data['Close'].rolling(window=50).mean()
        data['200_SMA'] = data['Close'].rolling(window=200).mean()
        return data

    def calculate_deviations(self, data):
        """Calculate the deviations of stock prices from 50D SMA and between 50D and 200D SMAs."""
        if data is not None and not data.empty:
            # Calculate deviation from 50-day SMA
            data['Price_Deviation_50SMA'] = abs(data['Close'] - data['50_SMA']) / data['50_SMA']

            # Calculate deviation between 50-day SMA and 200-day SMA
            data['SMA_Deviation'] = abs(data['50_SMA'] - data['200_SMA']) / data['200_SMA']

            return data
        return None

    def filter_stocks(self):
        """Filter stocks based on minimum price deviation from 50D SMA and maximum SMA deviation."""
        for stock in self.stock_list:
            print(f"Analyzing {stock}...")
            data = self.fetch_data(stock)
            if data is not None and not data.empty:
                data = self.calculate_smas(data)
                data = self.calculate_deviations(data)

                # Check for stocks that meet both criteria
                if data is not None and not data.empty:
                    # Check average price deviation from 50D SMA over the entire time period
                    avg_price_deviation = data['Price_Deviation_50SMA'].mean()

                    # Check maximum deviation between 50D SMA and 200D SMA
                    avg_sma_deviation = data['SMA_Deviation'].mean()

                    # Append to the list of filtered stocks if both criteria are met
                    self.filtered_stocks.append({
                        'stock': stock,
                        'avg_price_deviation': avg_price_deviation,
                        'avg_sma_deviation': avg_sma_deviation
                    })

    def get_filtered_stocks(self):
        """Return the filtered stocks sorted by min price deviation and max SMA deviation."""
        # Sort by min price deviation and avg SMA deviation
        filtered_stocks_sorted = sorted(
            self.filtered_stocks,
            key=lambda x: (x['avg_price_deviation'], -x['avg_sma_deviation'])
        )
        return filtered_stocks_sorted


In [4]:
# Define the list of stocks to analyze
stock_n100 = mypf['Symbol'].unique()

exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ']

stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

stock_list = stock_n100

# Define the list of stocks to analyze
#stock_list = ['TATAMOTORS.NS', 'RELIANCE.NS', 'INFY.NS', 'HDFCBANK.NS']

# Instantiate the StockSMADeviationFilter class
sma_filter = StockSMADeviationFilter(stock_list, start_date='2023-01-01')

# Filter stocks based on the criteria
sma_filter.filter_stocks()

# Get and print the filtered stocks
filtered_stocks = sma_filter.get_filtered_stocks()
for stock_info in filtered_stocks:
    print(f"Stock: {stock_info['stock']}, Avg Price Deviation: {stock_info['avg_price_deviation']:.4f}, Max SMA Deviation: {stock_info['avg_sma_deviation']:.4f}")



[*********************100%%**********************]  1 of 1 completed

Analyzing 3MINDIA...
Analyzing 5PAISA...



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Analyzing AARTIIND...
Analyzing AAVAS...
Analyzing ABBOTINDIA...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


...

Analyzing WESTLIFE...
Analyzing WHIRLPOOL...
Analyzing WIPRO...


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Analyzing WSTCSTPAPR...
Analyzing YESBANK...
Analyzing ZEEL...


[*********************100%%**********************]  1 of 1 completed


Stock: RELAXO, Avg Price Deviation: 0.0257, Max SMA Deviation: 0.0385
Stock: SBIETFPB, Avg Price Deviation: 0.0284, Max SMA Deviation: 0.0322
Stock: PGHH, Avg Price Deviation: 0.0321, Max SMA Deviation: 0.0649
Stock: ICICIBANK, Avg Price Deviation: 0.0323, Max SMA Deviation: 0.0602
Stock: KOTAKBANK, Avg Price Deviation: 0.0328, Max SMA Deviation: 0.0191
Stock: DABUR, Avg Price Deviation: 0.0334, Max SMA Deviation: 0.0277
Stock: NESTLEIND, Avg Price Deviation: 0.0336, Max SMA Deviation: 0.0568
Stock: HINDUNILVR, Avg Price Deviation: 0.0342, Max SMA Deviation: 0.0318
Stock: BAJAJFINSV, Avg Price Deviation: 0.0347, Max SMA Deviation: 0.0381
Stock: PIDILITIND, Avg Price Deviation: 0.0354, Max SMA Deviation: 0.0585
Stock: TCS, Avg Price Deviation: 0.0359, Max SMA Deviation: 0.0604
Stock: SBICARD, Avg Price Deviation: 0.0369, Max SMA Deviation: 0.0534
Stock: SHREECEM, Avg Price Deviation: 0.0377, Max SMA Deviation: 0.0354
Stock: JSWSTEEL, Avg Price Deviation: 0.0378, Max SMA Deviation: 0.050

In [14]:
filtered_stocks = pd.DataFrame(filtered_stocks)
filtered_stocks[(filtered_stocks['avg_price_deviation'] < 0.2)  & (filtered_stocks['avg_sma_deviation'] > 0.3)]
## 0.025, 0.50

,stock,avg_price_deviation,avg_sma_deviation
211,MCX,0.091011,0.337600
236,BIRLAMONEY,0.115731,0.307925
239,JPPOWER,0.120352,0.397203
240,HSCL,0.136858,0.368071
241,ARE&M,0.144822,0.504554
243,SUZLON,0.176849,0.444090


In [6]:
import yfinance as yf
import pandas as pd
import numpy as np

class StockTradingSignals:

    def __init__(self, stock_list, start_date='2023-01-01', end_date=None):
        self.stock_list = stock_list
        self.start_date = start_date
        self.end_date = end_date
        self.trading_signals = {}

    def fetch_data(self, ticker):
        """Fetch stock data from Yahoo Finance."""
        try:
            data = yf.download(ticker+ '.NS', start=self.start_date, end=self.end_date)
            return data
        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")
            return None

    def calculate_smas(self, data):
        """Calculate the 50-day and 200-day SMAs."""
        data['50_SMA'] = data['Close'].rolling(window=50).mean()
        data['200_SMA'] = data['Close'].rolling(window=200).mean()
        return data

    def generate_trading_signals(self, data):
        """Generate buy and sell signals based on SMA crossovers."""
        data['Signal'] = 0  # 0 for no action, 1 for buy, -1 for sell

        # Buy signal: when 50D SMA < 200D SMA and Close crosses above 50D SMA
        buy_condition = (data['Close'] > data['50_SMA']) & (data['50_SMA'] < data['200_SMA']) & (data['Close'].shift(1) <= data['50_SMA'].shift(1))

        # Sell signal: when 50D SMA > 200D SMA and Close crosses below 50D SMA
        sell_condition = (data['Close'] < data['50_SMA']) & (data['50_SMA'] > data['200_SMA']) & (data['Close'].shift(1) >= data['50_SMA'].shift(1))

        # Assign signals
        data.loc[buy_condition, 'Signal'] = 1
        data.loc[sell_condition, 'Signal'] = -1

        return data

    def analyze_stocks(self):
        """Analyze all stocks in the list and generate trading signals."""
        for stock in self.stock_list:
            print(f"Analyzing {stock}...")
            data = self.fetch_data(stock)
            if data is not None and not data.empty:
                data = self.calculate_smas(data)
                data = self.generate_trading_signals(data)
                self.trading_signals[stock] = data[['Close', '50_SMA', '200_SMA', 'Signal']]

    def get_trading_signals(self):
        """Return the generated trading signals for all stocks."""
        return self.trading_signals


In [16]:
# Define the list of stocks to analyze
stock_list = ['BIRLAMONEY'] #filtered_stocks['stock'].values

# Instantiate the StockTradingSignals class
trading_signals = StockTradingSignals(stock_list, start_date='2023-01-01')

# Analyze stocks and generate trading signals
trading_signals.analyze_stocks()

# Get and print the trading signals
signals = trading_signals.get_trading_signals()
for stock, signal_data in signals.items():
    print(f"\n{stock} Trading Signals:")
    print(signal_data[signal_data['Signal'] != 0])  # Print only rows where there is a buy/sell signal


[*********************100%%**********************]  1 of 1 completed

Analyzing BIRLAMONEY...

BIRLAMONEY Trading Signals:
                 Close      50_SMA    200_SMA  Signal
Date                                                 
2023-12-20  100.250000  104.041000   69.90550      -1
2024-02-09  115.750000  116.288000   81.82100      -1
2024-02-16  115.400002  117.080000   83.43925      -1
2024-02-21  115.199997  117.569000   84.42750      -1
2024-04-08  110.400002  110.761000   92.29100      -1
2024-04-15  105.050003  109.194000   93.33950      -1
2024-08-05  155.720001  157.850799  124.00495      -1
2024-08-14  156.740005  162.921400  126.34660      -1
